In [3]:
import keras
keras.__version__

'2.1.2'

Data downloaded from: https://www.kaggle.com/c/whale-categorization-playground

I renamed train.csv -> targets.csv and the train directory to kaggle_train/

In [4]:
import os

home_dir = os.getcwd()
fname = os.path.join(home_dir, 'targets.csv') # targets for both train and validation

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
lines = lines[:-1]

print(header)
print(len(lines))

['Image', 'Id']
9850


# Encoding the whale ids


In [19]:
import numpy as np
whale_ids = [line.split(',')[1] for line in lines]
whale_ids = set(whale_ids) # convert to set to remove duplicats
whale_ids = list(whale_ids) # convert back to list to make it ordered


# Am no longer using the whale2vec function, sinc the ImageDataGenerator automatically one-hot-encodes the 
# targets
whale_dict = {}
for i, whale in enumerate(whale_ids):
    vec = np.zeros(len(whale_ids))
    vec[i] = 1
    whale_dict[whale] = vec
    
def whale2vec(whale): # returns a unique one-hot encoded vector given 
    if whale in whale_dict.keys():
        return whale_dict[whale]
    else:
        print("whale not found. Returning new_whale vector")
        return whale_dict['new_whale']
               
vec = whale2vec(whale_ids[2])
print(len(whale_ids))
print(vec)

4251
[ 0.  0.  1. ...,  0.  0.  0.]


# Preprocessing images

In [6]:
import errno 

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

First we split the training images into a train and validation set, and then subdivide them into directories for each whale_id

In [7]:
import shutil
original_dataset_dir = os.path.join(home_dir, 'kaggle_train')

train_dir = os.path.join(home_dir, 'train')
mkdir_p(train_dir)

validation_dir = os.path.join(home_dir, 'validation')
mkdir_p(validation_dir)

# Let's then create a subdirectory for each whale_id in both the train and validation directories 
# so we can use the ImageDataGenerator magic function

for whale_id in whale_ids:
    mkdir_p(os.path.join(train_dir, whale_id))
    mkdir_p(os.path.join(validation_dir, whale_id))
    
# Copy first 7500 files into the appropriate whale directory in train dir    
for i in range(7500):
    pic = lines[i].split(',')[0]
    whale_id = lines[i].split(',')[1]
    src = os.path.join(original_dataset_dir, pic)
    whale_id_dir = os.path.join(train_dir, whale_id)
    dst = os.path.join(whale_id_dir, pic)
    shutil.copyfile(src, dst)
    
# copy the rest into the appropriate whale directory in validation dir    
for i in range(7500,len(lines)):
    pic = lines[i].split(',')[0]
    whale_id = lines[i].split(',')[1]
    src = os.path.join(original_dataset_dir, pic)
    whale_id_dir = os.path.join(validation_dir, whale_id)
    dst = os.path.join(whale_id_dir, pic)
    shutil.copyfile(src, dst)


In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

image_size = (180,180) #adjustable parameter for processed image_size. Run time should 

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to image_size - define above
        target_size=image_size,
        color_mode='rgb', # input images are RGB and grayscale, but we map them all onto 
                                # which should be suitable for whale tails anyway
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        # This is the target directory
        validation_dir,
        # All images will be resized to image_size - define above
        target_size=image_size,
        color_mode='rgb',
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 7500 images belonging to 4251 classes.
Found 2350 images belonging to 4251 classes.


In [9]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(image_size[0],image_size[1],3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(8192, activation='relu'))
model.add(layers.Dense(len(whale_ids), activation='softmax'))


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 178, 178, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 41, 41, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 128)       147584    
__________

In [11]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [57]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=375, # batch size is 20, 375 steps will get us through
    7500 images
      epochs=20,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/20
375/375 [==============================] - 787s 2s/step - loss: 8.0286 - acc: 0.0857 - val_loss: 8.1145 - val_acc: 0.0760
Epoch 2/20
375/375 [==============================] - 763s 2s/step - loss: 7.7825 - acc: 0.0860 - val_loss: 8.0932 - val_acc: 0.0760
Epoch 3/20
375/375 [==============================] - 771s 2s/step - loss: 7.6613 - acc: 0.0860 - val_loss: 8.5058 - val_acc: 0.0760
Epoch 4/20
262/375 [===================>..........] - ETA: 3:43 - loss: 7.4480 - acc: 0.0876

KeyboardInterrupt: 

### Using a pre-trained convolutional base


In [14]:
from keras.applications import Xception

conv_base = Xception(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size[0], image_size[1], 3))

83689472/83683744 [==============================] - 20s 0us/step


In [15]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 180, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 89, 89, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 89, 89, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 89, 89, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

The final feature map has shape `(6, 6, 2048)`. That's the feature on top of which we will stick a densely-connected classifier.

At this point, there are two ways we could proceed: 

* Running the convolutional base over our dataset, recording its output to a Numpy array on disk, then using this data as input to a 
standalone densely-connected classifier similar to those you have seen in the first chapters of this book. This solution is very fast and 
cheap to run, because it only requires running the convolutional base once for every input image, and the convolutional base is by far the 
most expensive part of the pipeline. However, for the exact same reason, this technique would not allow us to leverage data augmentation at 
all.
* Extending the model we have (`conv_base`) by adding `Dense` layers on top, and running the whole thing end-to-end on the input data. This 
allows us to use data augmentation, because every input image is going through the convolutional base every time it is seen by the model. 
However, for this same reason, this technique is far more expensive than the first one.

We will cover both techniques. Let's walk through the code required to set-up the first one: recording the output of `conv_base` on our 
data and using these outputs as inputs to a new model.

We will start by simply running instances of the previously-introduced `ImageDataGenerator` to extract images as Numpy arrays as well as 
their labels. We will extract features from these images simply by calling the `predict` method of the `conv_base` model.

In [25]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

home_dir = os.getcwd()
train_dir = os.path.join(home_dir, 'train')
validation_dir = os.path.join(home_dir, 'validation')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 6, 6, 2048))
    classes_count = len(whale_ids)
    labels = np.zeros(shape=(sample_count, len(whale_ids)))
    generator = datagen.flow_from_directory(
        directory,
        target_size=image_size,
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 7500)
validation_features, validation_labels = extract_features(validation_dir, len(lines) - 7500)


Found 7500 images belonging to 4251 classes.
Found 2350 images belonging to 4251 classes.


In [30]:
train_features = np.reshape(train_features,(7500, 6*6*2048))
validation_features = np.reshape(validation_features, (len(lines)-7500, 6*6*2048))

In [31]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=6*6*2048)) # we can play around with 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(whale_ids), activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               18874624  
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 4251)              1092507   
Total params: 19,967,131
Trainable params: 19,967,131
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=20,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 7500 samples, validate on 2350 samples
Epoch 1/20
2860/7500 [==========>...................] - ETA: 57s - loss: 7.3075 - acc: 0.0853